# Institulac 

In [68]:
import re
import json
import time
from datetime import datetime

import pandas as pd
pd.set_option("max_rows",100)
#pd.set_option("display.max_columns",100)
pd.set_option("max_colwidth",1000)

import helium as h
from selenium.common.exceptions import NoSuchElementException

In [69]:
# DICT CAT-PRODS-TAB
with open('dict_tables.json') as file_json:
    dict_tables=json.loads(file_json.read())

In [70]:
# login =
# name_ins =
# usser =
# passw=

# login
browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
sleep=0.8

#browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
time.sleep(sleep)
h.click('Consulte Aquí')

time.sleep(sleep)
h.write('UNIVERSIDAD DE ANTIOQUIA',into='Digite el nombre de la Institución') # name ins

time.sleep(sleep)
h.click('Buscar')

time.sleep(sleep)
h.click(browser.find_element_by_id('list_instituciones'))

time.sleep(sleep)

time.sleep(sleep)
h.select('seleccione una','UNIVERSIDAD DE ANTIOQUIA') # name_ins

time.sleep(sleep)
h.write('annyarango',into='Usuario')                  # user

time.sleep(sleep)
h.write('1@Silver', into='Contraseña')                # passw

time.sleep(sleep)
h.click(h.Button('Ingresar'))

# cookie injection
time.sleep(sleep)
# implementation cookie injection

# get current cookie and store
new_cookie=browser.get_cookies()[0]
    
# create new_cookie with time_expire
time_expire = (datetime(2022,1,1) - datetime(1970,1,1)).total_seconds()
new_cookie['expiry'] = int(time_expire)
    
# delete cookie sites
browser.delete_all_cookies()
    
# add new cookie
browser.add_cookie(new_cookie)

# navigation 1
time.sleep(sleep)
h.click('Aval')

time.sleep(sleep)
h.click('Avalar grupos')

time.sleep(sleep)
h.click('Grupos Avalados')

# -- end login --

# list of total groups
#select max results per page
h.wait_until(h.Text('Ver Reporte').exists)
h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'))

time.sleep(sleep)
h.select(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'),'100')

# catch 1: groups info [name, lider, cod,  link to producs]  
# schema
# empty df
# select max items per page
# while until end
# try:
    # catch table
    # preproces table
    # catch urls
    # add url colums
    # add df
    # click next page -> raise error
# except Nosuchelement:
    # break
    
# catch 1: list of groups
dfg=pd.DataFrame()
cont=True

while cont:
    
    try:
        # catch source
        time.sleep(sleep)
        source_g=browser.page_source
        
        # catch table
        time.sleep(sleep)
        df=pd.read_html(source_g, attrs={"id":"grupos_avalados"}, header=2)[0]
        
        # and preprocces it
        c=[x for x in df.columns if x.find('Unnamed:') == -1]
        dfgp=df[c][1:-1]
        print(dfgp.columns,dfgp.shape)
        
        # catch urls
        url=[a.get_attribute('href') for a in browser.find_elements_by_xpath('//table[@id="grupos_avalados"]//td[5]/a')]
        dfgp['Revisar'] = url
        dfg=dfg.append(dfgp)
        
        # click next page. this instruction rise error of the end. 
        h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//tr/td[3]/a'))
        
    except NoSuchElementException as e:
        
        print(e)
        print('out of cicle')
        break
        
    time.sleep(sleep)
    time.sleep(sleep)
    
dfg = dfg.reset_index(drop=True)
assert dfg.shape[0] == 324

Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (24, 4)
Message: Unable to locate element: //table[@id="grupos_avalados"]//tr/td[3]/a

out of cicle


In [71]:
print(dfg['Nombre del grupo'][0:1].values[0],'\n',dfg['Revisar'][:1].values[0])

print(dfg['Nombre del grupo'][-1:].values[0],'\n',dfg['Revisar'][-1:].values[0])

Grupo Tandem en Nano-bio-física 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000020456
Promoción de la Salud 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000001767


In [72]:
dfg.columns

Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object')

In [78]:
time.sleep(sleep*2)

LD = [] # 
LP = []
LR = [] 
for idx in dfg.index[13:14]:
    
    # create db
    DBG = {}
    # part info group
    print(dfg.loc[idx,'Nombre del grupo'])

    # specific group url
    time.sleep(sleep)
    url_group = dfg.loc[idx,'Revisar']

    # go to url group
    time.sleep(sleep)
    browser.get(url_group)

    # catch two tables: info grupo and  members
    source=browser.page_source

    #info
    l_info=pd.read_html(source, match='Nombre Grupo')
    info_g=l_info[3].pivot(columns=0,values=1)
    
    # STORE INFO_GROUP
    DBG['Info_group'] = info_g

    # members
    l_int = pd.read_html(source,attrs={'id':'tblIntegrantes'},header=2)
    mem_g=l_int[0]
    
    # STORE_MEMBERS
    DBG['Members'] =  mem_g

    # Products

    #time.sleep(sleep*5) # time time time !!!
    h.wait_until(lambda: browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]') is not None)
    h.click(browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]'))

    # products by belongs to  # time time time
    #time.sleep(sleep*7)       # time time time
    h.wait_until(lambda: browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]') is not None)
    h.click(browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]'))

    time.sleep(sleep)
    url_products=browser.current_url


    # map all products, store those id categories that amount is different to 0 and id products asociated.
    # make queries with combinations of categories and products
    # make urls with diferent combinations of quieries
    # go to each of urls
    # load page source
    # catch table ( or tables) asociated with categories and products
    # store tables

    report = ''

    list_of_prods =[] #[[cat,prod],[cat,prod]...]

    # map all products and get products and subs diff to cero
    for i in browser.find_elements_by_xpath('//div[@id="accordionCatgP"]/h3'):

        report += i.text + '\n' 
        report += i.get_attribute('id') + '\n'     

        time.sleep(sleep)
        h.click(i)
        
        # cat
        cat_ = int(re.findall(r'\d+',i.text)[0])
        
        # create cat key in dict, for estore diferents products by this categori: 'NC_': {'ART_E':TABLE,
        #                                                                                 'ART_IMP':TABLE}
        if cat_ > 0:
            DBG[i.get_attribute('id')] = {}
            
        
        for j in browser.find_elements_by_xpath('//div[@aria-labelledby="%s"]/h3' % i.get_attribute('id')):

            report += '\t' + j.text + '\n' 
            report += '\t' + j.get_attribute('id') + '\n'
            
            #prod
            pro_ = int(re.findall(r'\d+', j.text)[0])

            if cat_ > 0 and pro_ > 0:  
                
                list_of_prods.append([i.get_attribute('id'),j.get_attribute('id')])

        time.sleep(sleep) 
        # h.click(a)
        h.click(i)

    # print(report)
    # print('\n')
    # print('--------------------------------')
    time.sleep(sleep*2)
    
    tables=[]
    
    for p in range(len(list_of_prods)):

            # make query 
            query='categoriaP=%s&subcategoriaP=%s&aval=P' % (list_of_prods[p][0],list_of_prods[p][1])

            # make url query
            url_query = url_products.split('?')[0] + '?' + query + '&' + url_products.split('?')[1]

            # retrieve id asociated tables
            table_id = dict_tables[list_of_prods[p][0]][list_of_prods[p][1]]

            # go to url product by group
            time.sleep(sleep)
        
            browser.get(url_query)

            # load page
            time.sleep(sleep)
            page_source = browser.page_source

            # catch tables
            if isinstance(table_id,str): # case one table

                # catch title table
                
                title_table = browser.find_element_by_xpath('//div/p[@class="titulo_tabla"]').text 
                # cathc table
                #print(table_id)
                time.sleep(sleep*2)
                table = pd.read_html(page_source,attrs={'id':table_id}, header=2)[0][1:-1]

                # store table
                DBG[list_of_prods[p][0]][list_of_prods[p][1]] = {table_id:table}
                # ---- in building ----

            elif isinstance(table_id, list): # case multiple tables

                for i in range(len(table_id)):

                    # catch title specific table 
                    title_table = browser.find_elements_by_xpath('//div/p[@class="titulo_tabla"]')[i].text

                    # catch table
                    table = pd.read_html(page_source,attrs={'id':table_id[i]}, header=2)[0][1:-1]

                    # store table
                    DBG[list_of_prods[p][0]][list_of_prods[p][1]] = {table_id[i]:table}
                    # -----------
    LD.append(DBG)
    LP.append(list_of_prods)
    LR.append(report)

Colombia: tradiciones de la palabra


In [79]:
LP[0]

[['NC_P', 'ART_IMP_P'],
 ['NC_P', 'ART_ELE_P'],
 ['NC_P', 'CAP_LIB_P'],
 ['ASC_P', 'GEN_CONT_IMP_P'],
 ['ASC_P', 'EV_CIENT_P'],
 ['ASC_P', 'RED_P'],
 ['ASC_P', 'EDIC_P'],
 ['FRH_P', 'TES_PREG_P'],
 ['FRH_P', 'PROY_INV_DES_P'],
 ['FRH_P', 'PROY_INV_RESP_SOC_P']]

In [80]:
print(LR[0])

Nuevo Conocimiento (7)
NC_P
	Artículos publicados en revistas especializadas - Impresos (4)
	ART_IMP_P
	Artículos publicados en revistas especializadas - Electrónicos (1)
	ART_ELE_P
	Libros resultado de investigación (0)
	LIB_P
	Capítulos en Libro resultado de investigación (2)
	CAP_LIB_P
	Notas científicas (0)
	NOT_CIE_P
	Patentes (0)
	PAT_P
	Obras o productos de investigación creación en Artes, Arquitectura y Diseño (0)
	PRD_INV_ART_P
	Variedad Vegetal (0)
	VAR_VEG_P
	Nueva Raza Animal (0)
	VAR_ANI_P
	Poblaciones mejoradas de razas pecuarias (0)
	RAZ_PEC_P
	Traducciones Filológicas y Edición de Fuentes (0)
	TRA_FIL_P
Desarrollo Tecnológico e Innovación (0)
DTI_P
	Diseño industrial (0)
	DIS_IND_P
	Circuitos integrados (0)
	CIR_INT_P
	Software (0)
	SOFT_P
	Productos nutraceúticos (0)
	NUTRA_P
	Colecciones científicas (0)
	COL_CIENT_P
	Nuevos registros científicos (0)
	REG_CIENT_P
	Planta Piloto (0)
	PLT_PIL_P
	Prototipo Industrial (0)
	PRT_IND_P
	Secreto Industrial (0)
	SEC_IND_P
	Prot

In [81]:
len(LP) == len(LD)

True

In [82]:
LD

[{'Info_group': 0  NaN  (Fecha de última actualización del aval  06 de junio de 2019)  \
  0  NaN  (Fecha de última actualización del aval  06 de junio de 2019)   
  1  NaN                                                            NaN   
  2  NaN                                                            NaN   
  3  NaN                                                            NaN   
  4  NaN                                                            NaN   
  5  NaN                                                            NaN   
  6  NaN                                                            NaN   
  
  0                         Nombre Grupo        Nombre Líder  \
  0                                  NaN                 NaN   
  1                                  NaN                 NaN   
  2  Colombia: tradiciones de la palabra                 NaN   
  3                                  NaN                 NaN   
  4                                  NaN                 NaN   

In [85]:
datos = LD[0]

In [92]:
datos['NC_P']

{'ART_IMP_P': {'ART_P_TABLE':   Unnamed: 0  \
  1          1   
  2          2   
  3          3   
  4          4   
  
                                                                                                                                                Título del artículo  \
  1                                                                                  El cuento colombiano en la revista colombiana ilustrada El Gráfico (1925-1941)   
  2                                                                                          El catálogo de Pijao Editores (1972-2000): una editorial ¿alternativa¿   
  3                                                                                Ediciones Espiral y Editorial Iqueima (1944-1975). Una apuesta por la literatura   
  4  «Narrativa Colombiana Contemporánea» (Ediciones Tercer Mundo, 1962-1967): valor y cambio literario dentro del espacio editorial colombiano de los años sesenta   
  
    Año de presentación Mes de presentaci

In [62]:
dfg.loc[0,'COL Grupo']

'COL0202529'

In [61]:
dfg.columns

Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object')

In [90]:
idxx = 0
### build excels
name = 'AA_Plantilla_Formato de verificación de información_GrupLAC_894-2021_%s'

writer = pd.ExcelWriter(name % dfg.loc[idxx,'COL Grupo']+'.xlsx', engine='xlsxwriter')

DBG = LD[idxx]

### WORK BOOK

# INFO GROUP
df = pd.DataFrame(DBG['Info_group']) 

# WORKSHEET 0
df.to_excel(writer, sheet_name='Info_group')

# Members         
df = pd.DataFrame(DBG['Members'])

# WORKSHEET 1
df.to_excel(writer, sheet_name='Members')


### NC_P ### WORKSHEET 2
     
var_nc = 0

try:
    T=DBG['NC_P']['ART_IMP_P']['ART_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
        
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['ART_ELE_P']['ART_E_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['LIB_P']['LIB_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
        
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['CAP_LIB_P']['CAP_LIB_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
        
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['NOT_CIE_P']['NOT_CIE_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
        
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['PAT_P']['PAT_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['PRD_INV_ART_P']['PAAD_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
        
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['VAR_VEG_P']['VV_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
           
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['VAR_ANI_P']['VA_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
        
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['RAZ_PEC_P']['RAZ_PEC_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['NC_P']['TRA_FIL_P']['TRA_FIL_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    var_nc += df.shape[0]
    
except KeyError as e:
    
    pass

#### DTI_P

var_dt = 0
try:

    T=DBG['DTI_P']['DIS_IND_P']['DI_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['CIR_INT_P']['ECI_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['SOFT_P']['SF_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['NUTRA_P']['NUTRA_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['COL_CIENT_P']['COL_CIENT_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['REG_CIENT_P']['REG_CIENT_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['PLT_PIL_P']['PP_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['PRT_IND_P']['PI_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['SEC_IND_P']['SE_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['PROT_VIG_EPID_P']['PROT_VIG_EPID_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['EMP_BSE_TEC_P']['EBT_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['EMP_CRE_CUL_P']['ICC_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['INN_GES_EMP_P']['IG_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['INN_PROC_P']['IPP_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['REG_NORM_REGL_LEG_P']['RNR_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['CONP_TEC_P']['CONP_TEC_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['REG_AAD_P']['AAAD_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['DTI_P']['SIG_DIS_P']['SD_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    var_dt += df.shape[0]
    
except KeyError as e:
    
    pass

######  ASC

var_as = 0
try:
    T=DBG['ASC_P']['GEN_CONT_IMP_P']['GC_I_P_TABLE_5']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass                  

try:
    T=DBG['ASC_P']['PASC_P']['PASC_FOR_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['ASC_P']['PASC_P']['PASC_TRA_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['ASC_P']['PASC_P']['PASC_GEN_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['ASC_P']['PASC_P']['PASC_CAD_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['ASC_P']['DC_P']['DC_CD_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['ASC_P']['DC_P']['DC_CON_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['ASC_P']['DC_P']['DC_TRA_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['ASC_P']['DC_P']['DC_TRA_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='ASC_P',startrow = var_ac)
    
    var_as += df.shape[0]
    
except KeyError as e:
    
    pass

# FRH

var_rh = 0

try:
    T=DBG['FRH_P']['TES_DOC_P']['TD_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    var_rh += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['FRH_P']['TES_MAST_P']['TM_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    var_rh += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['FRH_P']['TES_PREG_P']['TP_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    var_rh += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['FRH_P']['ASE_PRG_ACA_P']['APGA_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    var_rh += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['FRH_P']['ASE_CRE_CUR_P']['ACC_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    var_rh += df.shape[0]
    
except KeyError as e:
    
    pass

try:
    T=DBG['FRH_P']['ASE_PRG_ONDAS_P']['APO_P_TABLE']
    
    df = pd.DataFrame(T)
    
    df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    var_rh += df.shape[0]
    
except KeyError as e:
    
    pass

In [91]:
writer.save()

In [17]:
str_ = "Direcciones de trabajo de grado de maestría (3)"

In [18]:
import re

In [21]:
int(re.findall(r'\d+',str_)[0])

3

In [37]:
'''
schema for store tables

DBG = {
    
    'INFO_GROUP': 'TABLE',
    'MEMBERS':'TABLE',
       
    'NC_P': {'ART_IMP_P': {'ART_P_TABLE':'TABLE'},
             'ART_ELE_P': {'ART_E_P_TABLE':'TABLE'},
             'LIB_P':     {'LIB_P_TABLE':'TABLE'},
             'CAP_LIB_P': {'CAP_LIB_P_TABLE':'TABLE'},
             'NOT_CIE_P': {'NOT_CIE_P_TABLE':'TABLE'},
             'PAT_P':     {'PAT_P_TABLE':'TABLE'},
             'PRD_INV_ART_P': {'PAAD_P_TABLE':'TABLE'},
             'VAR_VEG_P':     {'VV_P_TABLE':'TABLE'},
             'VAR_ANI_P':     {'VA_P_TABLE':'TABLE'},
             'RAZ_PEC_P':     {'RAZ_PEC_P_TABLE':'TABLE'},
             'TRA_FIL_P': {'TRA_FIL_P_TABLE':'TABLE'}
            },
     'DTI_P': {'DIS_IND_P': {'DI_P_TABLE':'TABLE'},
              'CIR_INT_P': {'ECI_P_TABLE':'TABLE'},
              'SOFT_P': {'SF_P_TABLE':'TABLE'},
              'NUTRA_P': {'NUTRA_P_TABLE':'TABLE'},
              'COL_CIENT_P': {'COL_CIENT_P_TABLE':'TABLE'},
              'REG_CIENT_P': {'REG_CIENT_P_TABLE':'TABLE'},
              'PLT_PIL_P': {'PP_P_TABLE':'TABLE'},
              'PRT_IND_P': {'PI_P_TABLE':'TABLE'},
              'SEC_IND_P': {'SE_P_TABLE':'TABLE'},
              'PROT_VIG_EPID_P': {'PROT_VIG_EPID_P_TABLE':'TABLE'},
              'EMP_BSE_TEC_P': {'EBT_P_TABLE':'TABLE'},
              'EMP_CRE_CUL_P': {'ICC_P_TABLE':'TABLE'},
              'INN_GES_EMP_P': {'IG_P_TABLE':'TABLE'},
              'INN_PROC_P': {'IPP_P_TABLE':'TABLE'},
              'REG_NORM_REGL_LEG_P': {'RNR_P_TABLE':'TABLE'},
              'CONP_TEC_P': {'CONP_TEC_P_TABLE':'TABLE'},
              'REG_AAD_P': {'AAAD_P_TABLE':'TABLE'},
              'SIG_DIS_P': {'SD_P_TABLE':'TABLE'}
              },
    'ASC_P': {'GEN_CONT_IMP_P': {'GC_I_P_TABLE_5':'TABLE'},
              'PASC_P': {'PASC_FOR_P_TABLE':'TABLE',
               'PASC_TRA_P_TABLE':'TABLE',
               'PASC_GEN_P_TABLE':'TABLE',
               'PASC_CAD_P_TABLE':'TABLE'},
              'DC_P': {'DC_CD_P_TABLE':'TABLE',
               'DC_CON_P_TABLE':'TABLE',
               'DC_TRA_P_TABLE':'TABLE',
               'DC_TRA_P_TABLE':'TABLE'}},
    'FRH_P': {'TES_DOC_P': {'TD_P_TABLE':'TABLE'},
              'TES_MAST_P': {'TM_P_TABLE':'TABLE'},
              'TES_PREG_P': {'TP_P_TABLE':'TABLE'},
              'PROY_INV_DES_P': {'PID_P_TABLE':'TABLE'},
              'PROY_INV_CRE_P': {'INV_CRE_P_TABLE':'TABLE'},
              'PROY_INV_DES_INN_P': {'PF_P_TABLE':'TABLE'},
              'PROY_INV_RESP_SOC_P': {'PE_P_TABLE':'TABLE'},
              'ASE_PRG_ACA_P': {'APGA_P_TABLE':'TABLE'},
              'ASE_CRE_CUR_P': {'ACC_P_TABLE':'TABLE'},
             'ASE_PRG_ONDAS_P': {'APO_P_TABLE':'TABLE'}}
}'''

In [14]:
a = 0
for i in [0,1,0,3,0,3,4,3]:  
    try:
        
        print(1/i)
        a += 1
    
    except ZeroDivisionError as e:
        
        pass
print(a)

1.0
0.3333333333333333
0.3333333333333333
0.25
0.3333333333333333
5


## --------- ##

In [22]:
# Create some Pandas dataframes from some data.
df1 = pd.DataFrame({'Data': [11, 12, 13, 14]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_positioningrr.xlsx', engine='xlsxwriter')

# Position the dataframes in the worksheet.

a=0

for i in [0,1,0,3,0,3,4,3]:  
    try:
        
        print(1/i)
        
        # Position the dataframes in the worksheet.
        
        df1.to_excel(writer, sheet_name='Sheet1', startrow=a+3)
        
        a += df1.shape[0]
    
    except ZeroDivisionError as e:
        
        pass
    
# Close the Pandas Excel writer and output the Excel file.
writer.save()

1.0
0.3333333333333333
0.3333333333333333
0.25
0.3333333333333333


In [21]:
a

20

In [ ]:
# store info group in dict
    # each key store list of tables related with

In [ ]:
# workbook structure: options
    # each worksheet is a categorie
        # store group in dict; store each table in dict structure 
        '''{
        
            info: table,
            members: table,
            products:{
                        NC:[tables],
                        DTI:[tables],
                        ASC:[tables],
                        FRH:[tables]
                        
            }
        }'''
    # each worksheet is a table product or tables of products

In [ ]:
# writing multiple dataframes to worksheets using pandas an
# XlsxWriter
# create dataFrames

# create a pandas excel writer using Xlsxwriter as the engine
writer = pd.ExcelWriter('name_excel.xlsx', engine='xlsxwriter')

# save in ram
df1.to_excel(writer, sheet_name='NAME')
df2.to_excel(writer, sheet_name='NAME')
df3.to_excel(writer,sheet_name='NAME')

# save in disk
writer.save()

In [ ]:
name = 'AA_Plantilla_Formato de verificación de información_GrupLAC_894-2021_%s'

# Positioning dataframes in a worksheet using pandas Xlswriter
import pandas as pd

# create dataframes
# ... df1, df2, df3, df4

# create a pandas excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_pos.xlsx', engine='xlsxwriter')

# position a pandas Excel writer using XlsWriter as the engine
# var_pos = 0; must be update with val of rows for last df 
df1.to_excel(writer,sheet_name='sheet_name')
var_pos+=df1.shape[0]
df2.to_excel(writer,sheet_name='sheet_name_1'start_row=var_pos)
var_pos+=df2.shape[0]